In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import time

# YOLO 신뢰도 임계값 및 거리 경고 임계값 설정
CONF_THRESHOLD = 0.3
DIST_THRESHOLD = 1200  # cm 단위 거리 경고 기준
FOCAL_LENGTH = 400  # 카메라 초점 거리
RESIZE_WIDTH = 1280
RESIZE_HEIGHT = 720

# 차선 영역을 나타내는 네 꼭짓점 비율 좌표 (좌측 하단 → 우측 하단 → 우측 상단 → 좌측 상단)
LANE_POLYGON_VERTICES = [
    (0.2, 1.0), (0.8, 1.0), (0.6, 0.6), (0.4, 0.6)
]

# 클래스별 실제 객체 높이 (cm)
KNOWN_HEIGHTS = {
    0: 160,   # 사람
    2: 150,   # 자동차
    3: 100,   # 오토바이
    5: 350,   # 버스
    7: 350    # 트럭
}

# 객체 종류별 표시 색상
CLASS_COLORS = {
    2: (0, 255, 0),    # 자동차
    5: (255, 255, 0),  # 버스
    7: (255, 0, 255)   # 트럭
}

# 유효 클래스 ID 리스트 생성
VALID_CLASS_IDS = list(KNOWN_HEIGHTS.keys())

# YOLO 모델 로드
model = YOLO(r"C:\Users\USER\Downloads\notyet\best.pt")

# 비디오 로드
video_path = r"C:\Users\USER\Downloads\notyet\police.mp4"
cap = cv2.VideoCapture(video_path)

# 결과 저장용 비디오 객체 생성
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"XVID"), 30, (RESIZE_WIDTH, RESIZE_HEIGHT))

# 차선 영역 폴리곤 좌표 계산 함수
def detect_lane_area(frame):
    h, w = frame.shape[:2]
    polygon = np.array([[(int(w * x), int(h * y)) for (x, y) in LANE_POLYGON_VERTICES]])
    return polygon[0]

# 원근 변환 행렬 계산 함수
def get_perspective_transform_matrix(frame):
    h, w = frame.shape[:2]
    src_pts = np.float32([[w * x, h * y] for (x, y) in LANE_POLYGON_VERTICES])
    dst_pts = np.float32([[0, h], [w, h], [w, 0], [0, 0]])
    return cv2.getPerspectiveTransform(src_pts, dst_pts)

# 경고 카운터 및 원근 변환 행렬 초기화
warning_counter = 3
M = None

# 배경이 있는 텍스트 출력 함수
def draw_text_with_background(img, text, org, font, scale, color, thickness):
    (tw, th), base = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    cv2.rectangle(img, (x, y - th - base), (x + tw + 4, y + base), (0, 0, 0), -1)
    cv2.putText(img, text, org, font, scale, color, thickness)

# 프레임 루프 시작
while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    # 최초 프레임에서 원근 변환 행렬 계산
    if M is None:
        M = get_perspective_transform_matrix(frame)

    # 객체 탐지 실행
    results = model(frame, conf=CONF_THRESHOLD, iou=0.5)
    annotated_frame = results[0].plot()  # 기본 탐지 시각화
    lane_polygon = detect_lane_area(frame)  # 차선 영역 폴리곤 계산
    collision_warning = False

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf.item())
        class_id = int(box.cls.item())
        pixel_height = y2 - y1

        # 너무 작은 객체는 무시
        if pixel_height < 5:
            continue

        # 객체 바닥 중심점 계산
        center_y = y2 - 0.2 * (y2 - y1)
        center = np.array([[(x1 + x2) / 2, center_y]], dtype=np.float32)
        is_inside = cv2.pointPolygonTest(lane_polygon, tuple(center[0]), False)

        if class_id in VALID_CLASS_IDS and conf > CONF_THRESHOLD and pixel_height > 20:
            # 중심점에 대해 원근 변환 수행
            center_warped = cv2.perspectiveTransform(np.array([center]), M)[0][0]

            # 거리 보정 계산
            known_height = KNOWN_HEIGHTS.get(class_id, 170)
            dist_pixel = (known_height * FOCAL_LENGTH) / pixel_height
            warped_y = center_warped[1]
            dist_y = DIST_THRESHOLD * (1 - warped_y / RESIZE_HEIGHT)
            dist_y = max(50, dist_y)

            # 최종 거리 (픽셀 기반, 보정 기반 가중 평균)
            distance_cm = dist_pixel * 0.7 + dist_y * 0.3
            distance_m = distance_cm / 100

            # 경고 여부 판단
            if distance_cm < DIST_THRESHOLD:
                collision_warning = True
                box_color = (0, 0, 255)
                thickness = 3
            else:
                box_color = CLASS_COLORS.get(class_id, (255, 255, 255))
                thickness = 2

            # 바운딩 박스 및 거리 표시
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, thickness)
            dist_label = f"{distance_m:.1f}m"
            draw_text_with_background(annotated_frame, dist_label, (x1, y2 + 25),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

            # 중심점 표시
            cv2.circle(annotated_frame, (int(center_warped[0]), int(center_warped[1])),
                       5, (255, 0, 0), -1)

    # 경고 텍스트 점등 지속 시간 조절
    warning_counter = min(warning_counter + 5, 30) if collision_warning else max(warning_counter - 1, 0)

    # 경고 시 텍스트 출력
    if warning_counter > 0:
        h, w = frame.shape[:2]
        draw_text_with_background(annotated_frame, "⚠ Forward Collision Warning ⚠",
                                  (int(w * 0.15), 60),
                                  cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # 차선 영역 폴리곤 시각화
    cv2.polylines(annotated_frame, [lane_polygon], isClosed=True,
                  color=(0, 255, 255), thickness=2)

    # 시점 변환된 차선을 다시 원래 좌표계로 투영 후 시각화
    lane_poly_np = np.array([lane_polygon], dtype=np.float32)
    warped_lane = cv2.perspectiveTransform(lane_poly_np, M)
    reprojected_lane = cv2.perspectiveTransform(warped_lane, np.linalg.inv(M))
    cv2.polylines(annotated_frame, [np.int32(reprojected_lane)], isClosed=True,
                  color=(0, 150, 255), thickness=2)

    # FPS 계산 및 표시
    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # 프레임 저장 및 출력
    out.write(annotated_frame)
    cv2.imshow("YOLO + Collision Warning + Warped Lane", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\USER\\Downloads\\notyet\\best.pt'

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import time

# 설정 상수
CONF_THRESHOLD = 0.3
DIST_THRESHOLD = 1200  # cm
FOCAL_LENGTH = 400
RESIZE_WIDTH = 1280
RESIZE_HEIGHT = 720

LANE_POLYGON_VERTICES = [
    (0.2, 1.0), (0.8, 1.0), (0.6, 0.6), (0.4, 0.6)
]

KNOWN_HEIGHTS = {
    0: 160,  # 사람
    2: 150,  # 자동차
    3: 100,  # 오토바이
    5: 350,  # 버스
    7: 350   # 트럭
}

CLASS_COLORS = {
    2: (0, 255, 0),
    5: (255, 255, 0),
    7: (255, 0, 255)
}

VALID_CLASS_IDS = list(KNOWN_HEIGHTS.keys())

model = YOLO("/home/hkit/Downloads/3team3/best.pt")

cap = cv2.VideoCapture("/home/hkit/Downloads/3team3/서성네거리 11중 추돌사고 당시 블랙박스 영상.mp4")
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"XVID"), 30, (RESIZE_WIDTH, RESIZE_HEIGHT))

def detect_lane_area(frame):
    h, w = frame.shape[:2]
    polygon = np.array([[(int(w * x), int(h * y)) for (x, y) in LANE_POLYGON_VERTICES]])
    return polygon[0]

def get_perspective_transform_matrix(frame):
    h, w = frame.shape[:2]
    src_pts = np.float32([[w * x, h * y] for (x, y) in LANE_POLYGON_VERTICES])
    dst_pts = np.float32([[0, h], [w, h], [w, 0], [0, 0]])
    return cv2.getPerspectiveTransform(src_pts, dst_pts)

def draw_text_with_background(img, text, org, font, scale, color, thickness):
    (tw, th), base = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    cv2.rectangle(img, (x, y - th - base), (x + tw + 4, y + base), (0, 0, 0), -1)
    cv2.putText(img, text, org, font, scale, color, thickness)

def process_detections(results, lane_polygon, M):
    collision_warning = False
    annotated_frame = results[0].plot()

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf.item())
        class_id = int(box.cls.item())
        pixel_height = y2 - y1

        if pixel_height < 5:
            continue

        center_y = y2 - 0.2 * (y2 - y1)
        center = np.array([[(x1 + x2) / 2, center_y]], dtype=np.float32)
        is_inside = cv2.pointPolygonTest(lane_polygon, tuple(center[0]), False)

        if class_id in VALID_CLASS_IDS and conf > CONF_THRESHOLD and pixel_height > 20:
            center_warped = cv2.perspectiveTransform(np.array([center]), M)[0][0]
            known_height = KNOWN_HEIGHTS.get(class_id, 170)
            dist_pixel = (known_height * FOCAL_LENGTH) / pixel_height
            warped_y = center_warped[1]
            dist_y = DIST_THRESHOLD * (1 - warped_y / RESIZE_HEIGHT)
            dist_y = max(50, dist_y)

            distance_cm = dist_pixel * 0.7 + dist_y * 0.3
            distance_m = distance_cm / 100

            # 원본 경고 조건 그대로 유지
            if distance_cm < DIST_THRESHOLD:
                collision_warning = True
                box_color = (0, 0, 255)
                thickness = 3
            else:
                box_color = CLASS_COLORS.get(class_id, (255, 255, 255))
                thickness = 2

            # 바운딩 박스 및 거리 표시 (항상 표시)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, thickness)
            dist_label = f"{distance_m:.1f}m"
            class_label = model.model.names[class_id] if hasattr(model.model, 'names') else str(class_id)
            draw_text_with_background(annotated_frame, dist_label, (x1, y2 + 25),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

            # 중심점 표시
            cv2.circle(annotated_frame, (int(center_warped[0]), int(center_warped[1])),
                       5, (255, 0, 0), -1)

    return annotated_frame, collision_warning

warning_counter = 3
M = None

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    if M is None:
        M = get_perspective_transform_matrix(frame)

    results = model(frame, conf=CONF_THRESHOLD, iou=0.5)
    lane_polygon = detect_lane_area(frame)

    annotated_frame, collision_warning = process_detections(results, lane_polygon, M)

    warning_counter = min(warning_counter + 5, 30) if collision_warning else max(warning_counter - 1, 0)

    if warning_counter > 0:
        h, w = frame.shape[:2]
        draw_text_with_background(annotated_frame, "⚠ Forward Collision Warning ⚠",
                                  (int(w * 0.15), 60),
                                  cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    cv2.polylines(annotated_frame, [lane_polygon], isClosed=True, color=(0, 255, 255), thickness=2)

    lane_poly_np = np.array([lane_polygon], dtype=np.float32)
    warped_lane = cv2.perspectiveTransform(lane_poly_np, M)
    reprojected_lane = cv2.perspectiveTransform(warped_lane, np.linalg.inv(M))
    cv2.polylines(annotated_frame, [np.int32(reprojected_lane)], isClosed=True, color=(0, 150, 255), thickness=2)

    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    out.write(annotated_frame)
    cv2.imshow("YOLO + Collision Warning + Warped Lane", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: '\\home\\hkit\\Downloads\\3team3\\best.pt'

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import time

# 설정 상수
CONF_THRESHOLD = 0.3
DIST_THRESHOLD = 1200  # cm
FOCAL_LENGTH = 400
RESIZE_WIDTH = 1280
RESIZE_HEIGHT = 720

LANE_POLYGON_VERTICES = [
    (0.2, 1.0), (0.8, 1.0), (0.6, 0.6), (0.4, 0.6)
]

KNOWN_HEIGHTS = {
    0: 160,  # 사람
    2: 150,  # 자동차
    3: 100,  # 오토바이
    5: 350,  # 버스
    7: 350   # 트럭
}

CLASS_COLORS = {
    2: (0, 255, 0),
    5: (255, 255, 0),
    7: (255, 0, 255)
}

VALID_CLASS_IDS = list(KNOWN_HEIGHTS.keys())

model = YOLO("/home/hkit/Downloads/3team3/best.pt")

cap = cv2.VideoCapture("joohyeok.mp4")
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"XVID"), 30, (RESIZE_WIDTH, RESIZE_HEIGHT))

def detect_lane_area(frame):
    h, w = frame.shape[:2]
    polygon = np.array([[(int(w * x), int(h * y)) for (x, y) in LANE_POLYGON_VERTICES]])
    return polygon[0]

def get_perspective_transform_matrix(frame):
    h, w = frame.shape[:2]
    src_pts = np.float32([[w * x, h * y] for (x, y) in LANE_POLYGON_VERTICES])
    dst_pts = np.float32([[0, h], [w, h], [w, 0], [0, 0]])
    return cv2.getPerspectiveTransform(src_pts, dst_pts)

def draw_text_with_background(img, text, org, font, scale, color, thickness):
    (tw, th), base = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    cv2.rectangle(img, (x, y - th - base), (x + tw + 4, y + base), (0, 0, 0), -1)
    cv2.putText(img, text, org, font, scale, color, thickness)

def process_detections(results, lane_polygon, M):
    collision_warning = False
    annotated_frame = results[0].plot()

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf.item())
        class_id = int(box.cls.item())
        pixel_height = y2 - y1

        if pixel_height < 5:
            continue

        center_y = y2 - 0.2 * (y2 - y1)
        center = np.array([[(x1 + x2) / 2, center_y]], dtype=np.float32)
        is_inside = cv2.pointPolygonTest(lane_polygon, tuple(center[0]), False)

        if class_id in VALID_CLASS_IDS and conf > CONF_THRESHOLD and pixel_height > 20:
            center_warped = cv2.perspectiveTransform(np.array([center]), M)[0][0]
            known_height = KNOWN_HEIGHTS.get(class_id, 170)
            dist_pixel = (known_height * FOCAL_LENGTH) / pixel_height
            warped_y = center_warped[1]
            dist_y = DIST_THRESHOLD * (1 - warped_y / RESIZE_HEIGHT)
            dist_y = max(50, dist_y)

            distance_cm = dist_pixel * 0.7 + dist_y * 0.3
            distance_m = distance_cm / 100

            # 원본 경고 조건 그대로 유지
            if distance_cm < DIST_THRESHOLD:
                collision_warning = True
                box_color = (0, 0, 255)
                thickness = 3
            else:
                box_color = CLASS_COLORS.get(class_id, (255, 255, 255))
                thickness = 2

            # 바운딩 박스 및 거리 표시 (항상 표시)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, thickness)
            dist_label = f"{distance_m:.1f}m"
            class_label = model.model.names[class_id] if hasattr(model.model, 'names') else str(class_id)
            draw_text_with_background(annotated_frame, dist_label, (x1, y2 + 25),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

            # 중심점 표시
            cv2.circle(annotated_frame, (int(center_warped[0]), int(center_warped[1])),
                       5, (255, 0, 0), -1)

    return annotated_frame, collision_warning

warning_counter = 3
M = None

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    if M is None:
        M = get_perspective_transform_matrix(frame)

    results = model(frame, conf=CONF_THRESHOLD, iou=0.5)
    lane_polygon = detect_lane_area(frame)

    annotated_frame, collision_warning = process_detections(results, lane_polygon, M)

    warning_counter = min(warning_counter + 5, 30) if collision_warning else max(warning_counter - 1, 0)

    if warning_counter > 0:
        h, w = frame.shape[:2]
        draw_text_with_background(annotated_frame, "⚠ Forward Collision Warning ⚠",
                                  (int(w * 0.15), 60),
                                  cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    cv2.polylines(annotated_frame, [lane_polygon], isClosed=True, color=(0, 255, 255), thickness=2)

    lane_poly_np = np.array([lane_polygon], dtype=np.float32)
    warped_lane = cv2.perspectiveTransform(lane_poly_np, M)
    reprojected_lane = cv2.perspectiveTransform(warped_lane, np.linalg.inv(M))
    cv2.polylines(annotated_frame, [np.int32(reprojected_lane)], isClosed=True, color=(0, 150, 255), thickness=2)

    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    out.write(annotated_frame)
    cv2.imshow("YOLO + Collision Warning + Warped Lane", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 352x608 1 bus- truck, 2 cars, 128.2ms
Speed: 4.2ms preprocess, 128.2ms inference, 3.7ms postprocess per image at shape (1, 3, 352, 608)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/hkit/.local/lib/python3.10/site-packages/cv2/qt/plugins"



0: 352x608 1 bus- truck, 2 cars, 94.9ms
Speed: 13.6ms preprocess, 94.9ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 4 cars, 83.6ms
Speed: 2.7ms preprocess, 83.6ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 4 cars, 69.3ms
Speed: 2.7ms preprocess, 69.3ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 4 cars, 83.1ms
Speed: 2.7ms preprocess, 83.1ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 3 cars, 70.7ms
Speed: 2.8ms preprocess, 70.7ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 3 cars, 79.5ms
Speed: 3.1ms preprocess, 79.5ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 4 cars, 78.5ms
Speed: 4.2ms preprocess, 78.5ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 4 cars,

In [ ]:
## 아무 차선인식 대입해보고 한 것##

from ultralytics import YOLO
import cv2
import numpy as np
import time

# 설정 상수
CONF_THRESHOLD = 0.3
DIST_THRESHOLD = 1200  # cm
FOCAL_LENGTH = 400
RESIZE_WIDTH = 1280
RESIZE_HEIGHT = 720

LANE_POLYGON_VERTICES = [
    (0.2, 1.0), (0.8, 1.0), (0.6, 0.6), (0.4, 0.6)
]

KNOWN_HEIGHTS = {
    0: 160,  # 사람
    2: 150,  # 자동차
    3: 100,  # 오토바이
    5: 350,  # 버스
    7: 350   # 트럭
}

CLASS_COLORS = {
    2: (0, 255, 0),
    5: (255, 255, 0),
    7: (255, 0, 255)
}

VALID_CLASS_IDS = list(KNOWN_HEIGHTS.keys())

model = YOLO("/home/hkit/Downloads/3team3/best.pt")

cap = cv2.VideoCapture("police.mp4")
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"XVID"), 30, (RESIZE_WIDTH, RESIZE_HEIGHT))

# 개선된 차선 인식 함수 (B안)
def detect_lane_area(frame):
    h, w = frame.shape[:2]
    roi = frame[int(h * 0.6):, :]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    white_mask = cv2.inRange(hsv, np.array([0, 0, 200]), np.array([180, 40, 255]))
    yellow_mask = cv2.inRange(hsv, np.array([10, 80, 100]), np.array([40, 255, 255]))
    mask = cv2.bitwise_or(white_mask, yellow_mask)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    edges = cv2.Canny(mask, 50, 120)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=60, minLineLength=50, maxLineGap=80)

    points = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            points.append((x1, y1 + int(h * 0.6)))
            points.append((x2, y2 + int(h * 0.6)))

    if len(points) < 6:
        return None

    hull = cv2.convexHull(np.array(points))
    return hull.reshape(-1, 2)

def get_perspective_transform_matrix(frame):
    h, w = frame.shape[:2]
    src_pts = np.float32([[w * x, h * y] for (x, y) in LANE_POLYGON_VERTICES])
    dst_pts = np.float32([[0, h], [w, h], [w, 0], [0, 0]])
    return cv2.getPerspectiveTransform(src_pts, dst_pts)

def draw_text_with_background(img, text, org, font, scale, color, thickness):
    (tw, th), base = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    cv2.rectangle(img, (x, y - th - base), (x + tw + 4, y + base), (0, 0, 0), -1)
    cv2.putText(img, text, org, font, scale, color, thickness)

def process_detections(results, lane_polygon, M):
    collision_warning = False
    annotated_frame = results[0].plot()

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf.item())
        class_id = int(box.cls.item())
        pixel_height = y2 - y1

        if pixel_height < 5:
            continue

        center_y = y2 - 0.2 * (y2 - y1)
        center = np.array([[(x1 + x2) / 2, center_y]], dtype=np.float32)
        is_inside = cv2.pointPolygonTest(lane_polygon, tuple(center[0]), False)

        if class_id in VALID_CLASS_IDS and conf > CONF_THRESHOLD and pixel_height > 20:
            center_warped = cv2.perspectiveTransform(np.array([center]), M)[0][0]
            known_height = KNOWN_HEIGHTS.get(class_id, 170)
            dist_pixel = (known_height * FOCAL_LENGTH) / pixel_height
            warped_y = center_warped[1]
            dist_y = DIST_THRESHOLD * (1 - warped_y / RESIZE_HEIGHT)
            dist_y = max(50, dist_y)

            distance_cm = dist_pixel * 0.7 + dist_y * 0.3
            distance_m = distance_cm / 100

            if distance_cm < DIST_THRESHOLD:
                collision_warning = True
                box_color = (0, 0, 255)
                thickness = 3
            else:
                box_color = CLASS_COLORS.get(class_id, (255, 255, 255))
                thickness = 2

            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, thickness)
            dist_label = f"{distance_m:.1f}m"
            class_label = model.model.names[class_id] if hasattr(model.model, 'names') else str(class_id)
            draw_text_with_background(annotated_frame, dist_label, (x1, y2 + 25),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
            cv2.circle(annotated_frame, (int(center_warped[0]), int(center_warped[1])), 5, (255, 0, 0), -1)

    return annotated_frame, collision_warning

# === 메인 루프 ===
warning_counter = 3
M = None

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    if M is None:
        M = get_perspective_transform_matrix(frame)

    results = model(frame, conf=CONF_THRESHOLD, iou=0.5)
    lane_polygon = detect_lane_area(frame)

    if lane_polygon is not None:
        annotated_frame, collision_warning = process_detections(results, lane_polygon, M)

        warning_counter = min(warning_counter + 5, 30) if collision_warning else max(warning_counter - 1, 0)

        if warning_counter > 0:
            draw_text_with_background(annotated_frame, "!!! Forward Collision Warning !!!",
                                      (int(RESIZE_WIDTH * 0.15), 60),
                                      cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

        cv2.polylines(annotated_frame, [lane_polygon], isClosed=True, color=(0, 255, 255), thickness=2)

        lane_poly_np = np.array([lane_polygon], dtype=np.float32)
        warped_lane = cv2.perspectiveTransform(lane_poly_np, M)
        reprojected_lane = cv2.perspectiveTransform(warped_lane, np.linalg.inv(M))
        cv2.polylines(annotated_frame, [np.int32(reprojected_lane)], isClosed=True, color=(0, 150, 255), thickness=2)

    else:
        annotated_frame = results[0].plot()
        warning_counter = max(warning_counter - 1, 0)

    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    out.write(annotated_frame)
    cv2.imshow("YOLO + Collision Warning + Warped Lane", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 352x608 2 bus- trucks, 4 cars, 214.2ms
Speed: 8.2ms preprocess, 214.2ms inference, 12.2ms postprocess per image at shape (1, 3, 352, 608)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/hkit/.local/lib/python3.10/site-packages/cv2/qt/plugins"



0: 352x608 2 bus- trucks, 3 cars, 87.7ms
Speed: 5.3ms preprocess, 87.7ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 2 bus- trucks, 3 cars, 154.2ms
Speed: 23.6ms preprocess, 154.2ms inference, 2.4ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 2 bus- trucks, 4 cars, 104.1ms
Speed: 6.8ms preprocess, 104.1ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 2 bus- trucks, 3 cars, 119.6ms
Speed: 8.2ms preprocess, 119.6ms inference, 6.3ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 2 bus- trucks, 6 cars, 88.8ms
Speed: 2.6ms preprocess, 88.8ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 5 cars, 90.2ms
Speed: 6.4ms preprocess, 90.2ms inference, 1.7ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 4 cars, 102.1ms
Speed: 2.3ms preprocess, 102.1ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 1 bus- truck, 3 cars, 88.2ms
Speed: 

In [ ]:
import time
import cv2
import numpy as np
from ultralytics import YOLO

# 설정 상수
CONF_THRESHOLD = 0.3
DIST_THRESHOLD = 1200  # cm
FOCAL_LENGTH = 400
RESIZE_WIDTH = 1280
RESIZE_HEIGHT = 720

LANE_POLYGON_VERTICES = [
    (0.2, 1.0), (0.8, 1.0), (0.6, 0.6), (0.4, 0.6)
]

KNOWN_HEIGHTS = {
    0: 160,  # 사람
    2: 150,  # 자동차
    3: 100,  # 오토바이
    5: 350,  # 버스
    7: 350   # 트럭
}

CLASS_COLORS = {
    2: (0, 255, 0),
    5: (255, 255, 0),
    7: (255, 0, 255)
}

VALID_CLASS_IDS = list(KNOWN_HEIGHTS.keys())
from pathlib import Path

resource_path = Path("..") / "resource"
model = YOLO(resource_path / "best.pt")
video_path = resource_path/ "harder_challenge_video.mp4"

cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"XVID"), 30, (RESIZE_WIDTH, RESIZE_HEIGHT))

# 경고 배너 이미지 불러오기 및 스케일 50% 축소
warning_banner = cv2.imread(resource_path /"warning_banner.png", cv2.IMREAD_UNCHANGED)
if warning_banner is not None:
    warning_banner = cv2.resize(warning_banner, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

# ======= 유틸 함수 =======

def draw_text_with_background(img, text, org, font, scale, color, thickness):
    (tw, th), base = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    cv2.rectangle(img, (x, y - th - base), (x + tw + 4, y + base), (0, 0, 0), -1)
    cv2.putText(img, text, org, font, scale, color, thickness)

def overlay_warning_banner(frame, banner_img, x, y):
    bh, bw = banner_img.shape[:2]
    fh, fw = frame.shape[:2]

    # 배너가 프레임 바깥일 경우 그리지 않음 (완전 밖으로 나가면 return)
    if x >= fw or x + bw <= 0:
        return
    if y >= fh or y + bh <= 0:
        return

    # 배너가 화면 밖으로 살짝 나가도 일부만 그릴 수 있도록 처리
    x1_frame = max(x, 0)
    y1_frame = max(y, 0)
    x1_banner = max(0, -x)
    y1_banner = max(0, -y)

    x2_frame = min(fw, x + bw)
    y2_frame = min(fh, y + bh)
    x2_banner = x2_frame - x
    y2_banner = y2_frame - y

    if banner_img.shape[2] == 4:  # 알파 채널 처리
        alpha = banner_img[y1_banner:y2_banner, x1_banner:x2_banner, 3] / 255.0
        for c in range(3):  # BGR
            frame[y1_frame:y2_frame, x1_frame:x2_frame, c] = (
                frame[y1_frame:y2_frame, x1_frame:x2_frame, c] * (1 - alpha) + 
                banner_img[y1_banner:y2_banner, x1_banner:x2_banner, c] * alpha
            ).astype(np.uint8)
    else:
        frame[y1_frame:y2_frame, x1_frame:x2_frame] = banner_img[y1_banner:y2_banner, x1_banner:x2_banner]


def detect_lane_area(frame):
    h, w = frame.shape[:2]
    roi = frame[int(h * 0.6):, :]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    white_mask = cv2.inRange(hsv, np.array([0, 0, 200]), np.array([180, 40, 255]))
    yellow_mask = cv2.inRange(hsv, np.array([10, 80, 100]), np.array([40, 255, 255]))
    mask = cv2.bitwise_or(white_mask, yellow_mask)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    edges = cv2.Canny(mask, 50, 120)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=60, minLineLength=50, maxLineGap=80)

    points = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            points.append((x1, y1 + int(h * 0.6)))
            points.append((x2, y2 + int(h * 0.6)))

    if len(points) < 6:
        return None

    hull = cv2.convexHull(np.array(points))
    return hull.reshape(-1, 2)

def get_perspective_transform_matrix(frame):
    h, w = frame.shape[:2]
    src_pts = np.float32([[w * x, h * y] for (x, y) in LANE_POLYGON_VERTICES])
    dst_pts = np.float32([[0, h], [w, h], [w, 0], [0, 0]])
    return cv2.getPerspectiveTransform(src_pts, dst_pts)

def process_detections(results, lane_polygon, M):
    collision_warning = False
    annotated_frame = results[0].plot()

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf.item())
        class_id = int(box.cls.item())
        pixel_height = y2 - y1

        if pixel_height < 5:
            continue

        center_y = y2 - 0.2 * (y2 - y1)
        center = np.array([[(x1 + x2) / 2, center_y]], dtype=np.float32)
        is_inside = cv2.pointPolygonTest(lane_polygon, tuple(center[0]), False)

        if class_id in VALID_CLASS_IDS and conf > CONF_THRESHOLD and pixel_height > 20:
            center_warped = cv2.perspectiveTransform(np.array([center]), M)[0][0]
            known_height = KNOWN_HEIGHTS.get(class_id, 170)
            dist_pixel = (known_height * FOCAL_LENGTH) / pixel_height
            warped_y = center_warped[1]
            dist_y = DIST_THRESHOLD * (1 - warped_y / RESIZE_HEIGHT)
            dist_y = max(50, dist_y)

            distance_cm = dist_pixel * 0.7 + dist_y * 0.3
            distance_m = distance_cm / 100

            if distance_cm < DIST_THRESHOLD:
                collision_warning = True
                box_color = (0, 0, 255)
                thickness = 3
            else:
                box_color = CLASS_COLORS.get(class_id, (255, 255, 255))
                thickness = 2

            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, thickness)
            dist_label = f"{distance_m:.1f}m"
            class_label = model.model.names[class_id] if hasattr(model.model, 'names') else str(class_id)
            draw_text_with_background(annotated_frame, dist_label, (x1, y2 + 25),
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
            cv2.circle(annotated_frame, (int(center_warped[0]), int(center_warped[1])), 5, (255, 0, 0), -1)

    return annotated_frame, collision_warning

# === 메인 루프 ===
warning_counter = 3
M = None

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    if M is None:
        M = get_perspective_transform_matrix(frame)

    results = model(frame, conf=CONF_THRESHOLD, iou=0.5)
    lane_polygon = detect_lane_area(frame)

    if lane_polygon is not None:
        annotated_frame, collision_warning = process_detections(results, lane_polygon, M)

        warning_counter = min(warning_counter + 5, 30) if collision_warning else max(warning_counter - 1, 0)

        if warning_counter > 0 and warning_banner is not None:
            banner_width = warning_banner.shape[1]
            x_pos = int((RESIZE_WIDTH - banner_width) / 2)  # 중앙 정렬
            y_pos = -90
            overlay_warning_banner(annotated_frame, warning_banner, x_pos, y_pos)


        cv2.polylines(annotated_frame, [lane_polygon], isClosed=True, color=(0, 255, 255), thickness=2)

        lane_poly_np = np.array([lane_polygon], dtype=np.float32)
        warped_lane = cv2.perspectiveTransform(lane_poly_np, M)
        reprojected_lane = cv2.perspectiveTransform(warped_lane, np.linalg.inv(M))
        cv2.polylines(annotated_frame, [np.int32(reprojected_lane)], isClosed=True, color=(0, 150, 255), thickness=2)

    else:
        annotated_frame = results[0].plot()
        warning_counter = max(warning_counter - 1, 0)

    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    out.write(annotated_frame)
    cv2.imshow("YOLO + Collision Warning + Warped Lane", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 352x608 (no detections), 81.5ms
Speed: 2.3ms preprocess, 81.5ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 (no detections), 44.6ms
Speed: 1.6ms preprocess, 44.6ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 (no detections), 44.7ms
Speed: 2.4ms preprocess, 44.7ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 (no detections), 40.2ms
Speed: 1.7ms preprocess, 40.2ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 (no detections), 39.1ms
Speed: 1.6ms preprocess, 39.1ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 (no detections), 38.9ms
Speed: 1.4ms preprocess, 38.9ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 (no detections), 36.7ms
Speed: 1.4ms preprocess, 36.7ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 608)

0: 352x608 (no detections), 39.7ms
Speed: 1.5ms preprocess, 39.7ms i

: 